In [1]:
import os
import zipfile
%load_ext autoreload

%autoreload 2

In [2]:
import tracemalloc
import psutil
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string
import time

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames
from listBundle.addAndUpdateListBundle.addAndUpdateListBundle import ListBundleHandler

class FolderNotFoundError(Exception):
    pass

class Metrics:
    
    def __init__(self, logFileName, logger):
        self.logFileName = logFileName
        self.start()
        self.writer = open(self.logFileName, 'a')
        self.writer.write("StepName,Time,Current Memory,Peak Memory,Used Ram Percentage\n")
        self.finalPeak = 0
        self.finalTotalTime = 0
        self.finalUsedRamPerc = 0
        self.logger = logger
    
    def start(self):
        self.startTime = time.time()
        tracemalloc.start()
    
    def getMetric(self, msg):
        
        self.endTime = time.time()
        
        self.totalTime = self.endTime - self.startTime
        
        
        current, peak = tracemalloc.get_traced_memory()
        current = current / 10**6
        peak = peak / 10**6
        
        usedRamPerc = psutil.virtual_memory()[2]
        
        self.finalPeak = max(self.finalPeak, peak)
        self.finalUsedRamPerc = max(self.finalUsedRamPerc, usedRamPerc)

        self.finalTotalTime = self.finalTotalTime + self.totalTime
        #self.finalTotalTime = round(self.finalTotalTime/60,3)
        
        outputString = f"{msg},{round(self.totalTime/60,4)} Min,{current} MB,{peak} MB,{usedRamPerc}%\n"
        
        self.logger.logFlowCheckpoint(f"{outputString}")
        
        print(f"Metrics : {outputString}")
        self.writer.write(outputString)
        tracemalloc.stop()
        tracemalloc.start()
        self.startTime = time.time()
    def end(self):
        
        current, peak = tracemalloc.get_traced_memory()
        current = current / 10**6
        outputString = f"Final Metrics,{round(self.finalTotalTime/60,4)} Min,{current} MB,{self.finalPeak} MB,{self.finalUsedRamPerc}%\n"
        print(f"Metrics : {outputString}")
        self.logger.logFlowCheckpoint(f"{outputString}")
        self.writer.write(outputString)
        self.writer.close()
        tracemalloc.stop()
        
        


def convertToInt(x):
    try:
        return str(int(x))
    except:
        return x


def convertCollectionToDataFrame(collection):

    dfExtractedHier = pd.DataFrame(collection)
    dfExtractedHier['parent_id'] = dfExtractedHier['parent_id'].apply(
        lambda x: convertToInt(x))
    dfExtractedHier['id'] = dfExtractedHier['id'].apply(
        lambda x: convertToInt(x))

    return dfExtractedHier

def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog):

    module_path = os.path.join(basePath)

    if "/" in basePath:
        pathSep = "/"
    else:
        pathSep = "\\"
    
    # Generate output folder path
    output_json_path = os.path.join(basePath, 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, htmlDocName))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', htmlDocName,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                             controlBasePath=controlBasePath,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType
                                             )

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)

        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = os.path.join(output_json_path, htmlDocName)
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
            
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                        controlBasePath=controlBasePath,
                                        language=languageCode,
                                        fileName=fileNameQrd,
                                        domain=domain,
                                        procedureType=procedureType
                                        )
    
    path_json = os.path.join(basePath,'outputJSON', fileNameJson)
    print("PathJson",path_json)
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(controlBasePath,
                                basePath,
                                domain,
                                procedureType,
                                languageCode,
                                documentNumber,
                                fileNameDoc,
                                fileNameQrd,
                                fileNameMatchRuleBook,
                                fileNameDocumentTypeNames,
                                fileNameLog,
                                stopWordFilterLen=6,
                                isPackageLeaflet=False,
                                medName=None
                                ):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}_{getRandomString(1)}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        controlBasePath=controlBasePath,
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber
        ).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        controlBasePath,
        basePath,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName)
    df, coll, documentType = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll, documentType


def parseDocument(controlBasePath,
                  basePath,
                  htmlDocName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName = None):
    
    listRegulatedAuthCodesAccrossePI = []
    
    if "/" in basePath:
        pathSep = "/"        
    else:
        pathSep = "\\"
    
    fileNameLog = os.path.join(basePath,'FinalLog.txt')

    pathComponents = basePath.split(pathSep)
    print(pathComponents, htmlDocName)
    timestamp = pathComponents[-1]
    languageCode =  pathComponents[-2]
    medName = pathComponents[-3]
    procedureType = pathComponents[-4]
    domain = pathComponents[-5]

    print(timestamp, languageCode, medName, procedureType, domain)
        
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)
    
    metrics = Metrics(os.path.join(basePath,'Metrics.csv'),flowLogger)
    
    
    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    fileNameJson, stylesFilePath = convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog)
    
    print("stylePath:-",stylesFilePath)
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")
    metrics.getMetric("HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    metrics.getMetric("Split Json")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        #print("Index", index)
        #if index in [2,3]:
        #    continue
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll, documentType = extractAndValidateHeadings(controlBasePath,
                                    basePath,
                                    domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName)
    
        
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        metrics.getMetric(f"{index}: Heading Extraction")

        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll, index)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except Exception as e:
            pms_oms_annotation_data = None
            print("Error Found", str(e))
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        metrics.getMetric(f"{index}: Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, basePath, coll)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        metrics.getMetric(f"{index}: Content Extraction")
        
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, controlBasePath, basePath, pms_oms_annotation_data, stylesFilePath, medName)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        metrics.getMetric(f"{index}: Generate XML")
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        
        fhirServiceObj = FhirService(fhirServiceLogger, apiMmgtBaseUrl, addBundleApiEndPointUrlSuffix, apiMmgtSubsKey, basePath, generatedXml)
        fhirServiceObj.submitFhirXml()
        
        
        
        
        metrics.getMetric(f"{index}: Submit FHIR Msg")
        
        print(f"Created XML File For :- {fileNamePartitioned}")
        
        flowLogger.logFlowCheckpoint("Starting list bundle update/addition")
        if documentAnnotationObj.listRegulatedAuthorizationIdentifiers != None:
            for id in documentAnnotationObj.listRegulatedAuthorizationIdentifiers:
                listRegulatedAuthCodesAccrossePI.append(id)
        listBundleLogger =  MatchLogger(f'List Bundle Creation Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        print("\nlistRegulatedAuthCodesAccrossePI",listRegulatedAuthCodesAccrossePI)
        try:
            listBundleHandler = ListBundleHandler(listBundleLogger,
                     domain,
                     procedureType,
                     index,
                     documentType,
                     languageCode,
                     medName,
                     controlBasePath,
                     jsonTempFileName,
                     listBundleDocumentTypeCodesFileName,
                     fileNameDocumentTypeNames,
                     listRegulatedAuthCodesAccrossePI,
                     apiMmgtBaseUrl,
                     getListApiEndPointUrlSuffix,
                     addUpdateListApiEndPointUrlSuffix,
                     apiMmgtSubsKey)

            listBundleXml = listBundleHandler.addOrUpdateDocumentItem(str(fhirServiceObj.SubmittedFhirMsgRefId))
            listBundleHandler.submitListXmLToServer(listBundleXml)

            flowLogger.logFlowCheckpoint("Completed list bundle update/addition")
            metrics.getMetric(f"{index}: Update/Add List Bundle")
            #return df,coll,dfExtractedHierRR
        except Exception as e:
            print(str(e))
            if 'No MAN Code found' in str(e):
                flowLogger.logFlowCheckpoint("Skipping list bundle addtion/update as no MAN found")
            
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")
    metrics.getMetric(f"{index}: Completed")
    metrics.end()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\psaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from wordToHtmlConvertor.wordToHtmlConvertor import WordToHtmlConvertor

wordToHtmlConvertorObj = WordToHtmlConvertor()
wordToHtmlConvertorObj.convertWordToHTML()

In [3]:
def runAll(inputList):
    for inputDoc in inputList:
        # inputZipFolderPath = "F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\\inputblob"
        inputZipFolderPath = os.path.abspath(os.path.join('..'))
        inputZipFolderPath = os.path.join(inputZipFolderPath, 'inputblob')
        inputZipFileName = inputDoc

        fileNameQrd = 'qrd_canonical_model.csv'
        fileNameMatchRuleBook = 'ruleDict.json'
        fileNameDocumentTypeNames = 'documentTypeNames.json'
        fsMountName = '/mounted'
        jsonTempFileName = 'listBundleJsonTemplate.json'
        listBundleDocumentTypeCodesFileName = 'listBundleDocumentTypeCodes.json'
        apiMmgtBaseUrl = "https://ema-dap-epi-dev-fhir-apim.azure-api.net"
        getListApiEndPointUrlSuffix = "/epi/v1/List"
        addUpdateListApiEndPointUrlSuffix = "/epi-w/v1/List"
        addBundleApiEndPointUrlSuffix = "/epi-w/v1/Bundle"
        apiMmgtSubsKey = "ba6d7e9a73ed4facaa58fc983bf6db50"



        info = inputZipFileName.split("~")

        try:
            medName = info[0]
            domain = info[1]
            procedureType = info[2]
            languageCode = info[3]
            timestamp = info[4]
            timestamp = timestamp.replace(".zip","")

        except Exception:
            raise f"Missing required info in the zip file name {inputZipFileName}"

        if "\\" in os.getcwd():
            localEnv = True
            inputZipFolderPath = os.path.join(os.path.abspath(os.path.join('..')),inputZipFolderPath)
            outputFolderPath = os.path.join(os.path.abspath(os.path.join('..')), 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(os.path.abspath(os.path.join('..')),'control')
        else:
            localEnv = False
            inputZipFolderPath = os.path.join(f'{fsMountName}',inputZipFolderPath)
            outputFolderPath = os.path.join(f'{fsMountName}', 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(f'{fsMountName}','control')


        print(inputZipFileName, inputZipFolderPath, outputFolderPath, controlFolderPath)

        mode = 0o666

        if localEnv is True:
            inputZipFolderPath = inputZipFolderPath.replace("/","\\")
            outputFolderPath = outputFolderPath.replace("/","\\")
            controlFolderPath = controlFolderPath.replace("/","\\")

        try:
            os.makedirs(inputZipFolderPath, mode)
            os.makedirs(outputFolderPath, mode)
            os.makedirs(controlFolderPath, mode)

        except Exception:
            print("Already Present")

        with zipfile.ZipFile(f'{inputZipFolderPath}/{inputZipFileName}',"r") as zip_ref:
                zip_ref.extractall(outputFolderPath)


        _,_,fileNames = next(os.walk(outputFolderPath))
        htmlFileName = [fileName for fileName in fileNames if ".htm" in fileName][0]

        print(htmlFileName)

        parseDocument(controlFolderPath,
                  outputFolderPath,
                  htmlFileName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName)


In [ ]:
inputList = ['GONAL-f~H~CAP~en~2021-06-04T08-55-59Z.zip',
    'GONAL-f~H~CAP~es~2021-06-06T11-29-47Z.zip',
    'GONAL-f~H~CAP~de~2021-06-07T06-58-30Z.zip']
runAll(inputList)

In [ ]:
inputList = ['Ovaleap~H~CAP~de~2021-06-07T08-05-21Z.zip',
            'Ovaleap~H~CAP~en~2021-06-04T08-18-55Z.zip',
            'Ovaleap~H~CAP~es~2021-06-07T11-37-57Z.zip']
runAll(inputList)

In [ ]:
fp = open(os.path.join('F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\work\H\CAP\Ovaleap\el\\2021-06-07T09-07-39Z\Ovaleap_clean.htm'), 'r')

In [ ]:
with open(os.path.join('F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\work\H\CAP\Ovaleap\el\\2021-06-07T09-07-39Z\Ovaleap_clean.htm'), 'rb') as fp:
    soup = BeautifulSoup(fp, "html.parser")

    soup = cleanHTML(soup)
    soup.body['id']=uuid.uuid4()

In [ ]:
soup.find_all('b')

In [ ]:
from bs4 import NavigableString, BeautifulSoup

In [ ]:
soup = BeautifulSoup(fp, "html.parser")


In [ ]:
def cleanHTML(soup):
    """
        Function to remove empty tags in HTML
    """
    remove_tags_in = ['span', 'b', 'em', 'i', 'u']
    for tag in remove_tags_in:
        for tag_dom in soup.body.find_all(tag):
            print("text:-",tag_dom.text)
            dom_children = tag_dom.findChildren()
            if(len(dom_children)==0 ):
                text_in_tag = "".join([ text.replace("\n"," ").replace('\t',"").replace("\r","") for text in tag_dom.find_all(text=True, recursive=True)])
                if(text_in_tag.isspace() or len(text_in_tag)==0):
                    parent = tag_dom.parent
                    tag_dom.decompose()
                else:
                    tag_dom.string.replace_with(text_in_tag)
                
    return soup

In [ ]:
type(soup)

In [ ]:
soup.body.find_all('p')

In [ ]:
soup.body.find_all('p')

In [ ]:
inputList = ['Ovaleap~H~CAP~el~2021-06-07T09-07-39Z.zip']
runAll(inputList)

In [ ]:
inputList = ['Ovaleap~H~CAP~el~2021-06-07T09-07-39Z.zip']
runAll(inputList)

In [ ]:
inputList = ['GONAL-f~H~CAP~el~2021-06-06T10-02-52Z.zip']
runAll(inputList)

In [ ]:
inputList = ['GONAL-f~H~CAP~el~2021-06-06T10-02-52Z.zip']
runAll(inputList)

In [ ]:
inputList = ['Stalevo~H~CAP~el~2021-06-03T19-24-04Z.zip']
runAll(inputList)

In [ ]:

inputList = ['Stalevo~H~CAP~en~2021-06-04T02-38-15Z.zip']
runAll(inputList)

In [ ]:
inputList = ['Stalevo~H~CAP~el~2021-06-03T19-24-04Z.zip']
runAll(inputList)


In [6]:
inputList = ['Elocta~H~CAP~bg~2021-06-09T08-18-48Z.zip']
runAll(inputList)


2021-06-09 18:07:31,458 : Flow Logger HTML_7 : Starting HTML Conversion To Json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:07:31,470 : Style Dictionary_B : Creating default style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:07:31,546 : Style Dictionary_B : Qrd Section Keys Retrieved For Style Dictionary: КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА, ПРИЛОЖЕНИЕ II, A. ДАННИ ВЪРХУ ОПАКОВКАТА, Б.  ЛИСТОВКА | H | CAP |  bg | HTML | Elocta_clean.htm


Elocta~H~CAP~bg~2021-06-09T08-18-48Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'bg', '2021-06-09T08-18-48Z'] Elocta_clean.htm
2021-06-09T08-18-48Z bg Elocta CAP H
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.json


2021-06-09 18:07:33,978 : Parser_N : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.txt | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:07:40,531 : Parser_N : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:07:41,108 : Flow Logger HTML_7 : Completed HTML Conversion To Json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:07:41,110 : Flow Logger HTML_7 : HTML Conversion To Json,0.1609 Min,8.54179 MB,18.89327 MB,48.5%
 | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:07:41,124 : Flow Logger HTML_7 : Starting Json Split | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:07:41,130 : Style Dictionary_F : Reading style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | Json | Elocta_cle

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1609 Min,8.54179 MB,18.89327 MB,48.5%

PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.json


2021-06-09 18:07:41,710 : Partition_Q : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  bg | Json | Elocta_clean.json
2021-06-09 18:07:41,868 : Partition_Q : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\partitionedJSONs\Elocta_clean_ПРИЛОЖЕНИЕ II.json | H | CAP |  bg | Json | Elocta_clean.json
2021-06-09 18:07:42,066 : Partition_Q : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\partitionedJSONs\Elocta_clean_ ДАННИ ВЪРХУ ОПАКОВКАТА.json | H | CAP |  bg | Json | Elocta_clean.json
2021-06-09 18:07:42,104 : Partition_Q : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\bg\2021-06-09T08-18-48Z\partitionedJSONs\Elocta_clean_Б.  ЛИСТО

...

POST sucessful: XML added with id 1bc35de5-d0ab-453a-bba5-8fc99b177ec2
Metrics : 3: Submit FHIR Msg,0.0713 Min,0.044539 MB,2.267365 MB,48.8%

Created XML File For :- Elocta_clean_Б.  ЛИСТОВКА.json

listRegulatedAuthCodesAccrossePI ['EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008', 'EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008']


2021-06-09 18:09:18,978 : List Bundle Creation Logger_3_P : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  bg | 3 | Elocta_clean_Б.  ЛИСТОВКА.json
2021-06-09 18:09:19,766 : List Bundle Creation Logger_3_P : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  bg | 3 | Elocta_clean_Б.  ЛИСТОВКА.json
2021-06-09 18:09:20,555 : List Bundle Creation Logger_3_P : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  bg | 3 | Elocta_clean_Б.  ЛИСТОВКА.json
2021-06-09 18:09:21,370 : List Bundle Creation Logger_3_P : Getting list bundle for MAN EU/1/15/1046/005  | H | CAP |  bg | 3 | Elocta_clean_Б.  ЛИСТОВКА.json
2021-06-09 18:09:22,196 : List Bundle Creation Logger_3_P : Getting list bundle for MAN EU/1/15/1046/006  | H | CAP |  bg | 3 | Elocta_clean_Б.  ЛИСТОВКА.json
2021-06-09 18:09:23,013 : List Bundle Creation Logger_3_P : Getting list bundle for MAN EU/1/15/1046/007  | H | CAP |  bg | 3 | Elocta_clean_Б.  ЛИСТОВКА.json
2021-06-09 18:09:24,356 : List Bundle Creation

Updating


2021-06-09 18:09:33,406 : List Bundle Creation Logger_3_P : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  bg | 3 | Elocta_clean_Б.  ЛИСТОВКА.json
2021-06-09 18:09:33,408 : Flow Logger HTML_7 : Completed list bundle update/addition | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:09:33,410 : Flow Logger HTML_7 : 3: Update/Add List Bundle,0.254 Min,0.276558 MB,0.956367 MB,48.3%
 | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:09:33,413 : Flow Logger HTML_7 : Completed Processing Partitioned Jsons | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:09:33,415 : Flow Logger HTML_7 : 3: Completed,0.0 Min,0.149606 MB,0.152259 MB,48.3%
 | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-09 18:09:33,417 : Flow Logger HTML_7 : Final Metrics,2.031 Min,0.0 MB,18.89327 MB,48.8%
 | H | CAP |  bg | HTML | Elocta_clean.htm


Metrics : 3: Update/Add List Bundle,0.254 Min,0.276558 MB,0.956367 MB,48.3%

Metrics : 3: Completed,0.0 Min,0.149606 MB,0.152259 MB,48.3%

Metrics : Final Metrics,2.031 Min,0.0 MB,18.89327 MB,48.8%



In [7]:
inputList = ['Elocta~H~CAP~cs~2021-06-09T09-32-50Z.zip']
runAll(inputList)


2021-06-09 18:09:33,671 : Flow Logger HTML_e : Starting HTML Conversion To Json | H | CAP |  cs | HTML | Elocta_clean.htm
2021-06-09 18:09:33,681 : Style Dictionary_k : Creating default style dictionary in file: rule_dictionary_cs.json | H | CAP |  cs | HTML | Elocta_clean.htm
2021-06-09 18:09:33,741 : Style Dictionary_k : Qrd Section Keys Retrieved For Style Dictionary: SOUHRN ÚDAJŮ O PŘÍPRAVKU, PŘÍLOHA II, A. OZNAČENÍ NA OBALU, B. PŘÍBALOVÁ INFORMACE | H | CAP |  cs | HTML | Elocta_clean.htm


Elocta~H~CAP~cs~2021-06-09T09-32-50Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'cs', '2021-06-09T09-32-50Z'] Elocta_clean.htm
2021-06-09T09-32-50Z cs Elocta CAP H
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\outputJSON\Elocta_clean.json


2021-06-09 18:09:35,232 : Parser_v : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\outputJSON\Elocta_clean.txt | H | CAP |  cs | HTML | Elocta_clean.htm
2021-06-09 18:09:41,144 : Parser_v : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\outputJSON\Elocta_clean.json | H | CAP |  cs | HTML | Elocta_clean.htm
2021-06-09 18:09:41,652 : Flow Logger HTML_e : Completed HTML Conversion To Json | H | CAP |  cs | HTML | Elocta_clean.htm
2021-06-09 18:09:41,655 : Flow Logger HTML_e : HTML Conversion To Json,0.1331 Min,6.700804 MB,16.530249 MB,48.4%
 | H | CAP |  cs | HTML | Elocta_clean.htm
2021-06-09 18:09:41,666 : Flow Logger HTML_e : Starting Json Split | H | CAP |  cs | HTML | Elocta_clean.htm
2021-06-09 18:09:41,673 : Style Dictionary_J : Reading style dictionary in file: rule_dictionary_cs.json | H | CAP |  cs | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1331 Min,6.700804 MB,16.530249 MB,48.4%

PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\outputJSON\Elocta_clean.json


2021-06-09 18:09:42,162 : Partition_y : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  cs | Json | Elocta_clean.json
2021-06-09 18:09:42,324 : Partition_y : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\partitionedJSONs\Elocta_clean_PŘÍLOHA II.json | H | CAP |  cs | Json | Elocta_clean.json
2021-06-09 18:09:42,484 : Partition_y : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\partitionedJSONs\Elocta_clean_ OZNAČENÍ NA OBALU.json | H | CAP |  cs | Json | Elocta_clean.json
2021-06-09 18:09:42,513 : Partition_y : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\cs\2021-06-09T09-32-50Z\partitionedJSONs\Elocta_clean_ PŘÍBALOVÁ INFORM

Metrics : Split Json,0.0147 Min,0.351551 MB,18.059147 MB,48.4%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'cs'

In [8]:
inputList = ['Elocta~H~CAP~da~2021-06-09T09-34-52Z.zip']
runAll(inputList)


2021-06-09 18:10:56,121 : Flow Logger HTML_y : Starting HTML Conversion To Json | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:10:56,133 : Style Dictionary_x : Creating default style dictionary in file: rule_dictionary_da.json | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:10:56,224 : Style Dictionary_x : Qrd Section Keys Retrieved For Style Dictionary: PRODUKTRESUMÉ, BILAG II, A. ETIKETTERING , B. INDLÆGSSEDDEL | H | CAP |  da | HTML | Elocta_clean.htm


Elocta~H~CAP~da~2021-06-09T09-34-52Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'da', '2021-06-09T09-34-52Z'] Elocta_clean.htm
2021-06-09T09-34-52Z da Elocta CAP H
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\outputJSON\Elocta_clean.json


2021-06-09 18:10:57,620 : Parser_c : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\outputJSON\Elocta_clean.txt | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:03,454 : Parser_c : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\outputJSON\Elocta_clean.json | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:03,950 : Flow Logger HTML_y : Completed HTML Conversion To Json | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:03,952 : Flow Logger HTML_y : HTML Conversion To Json,0.1305 Min,6.919257 MB,16.650975 MB,48.8%
 | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:03,967 : Flow Logger HTML_y : Starting Json Split | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:03,973 : Style Dictionary_z : Reading style dictionary in file: rule_dictionary_da.json | H | CAP |  da | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1305 Min,6.919257 MB,16.650975 MB,48.8%

PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\outputJSON\Elocta_clean.json


2021-06-09 18:11:04,408 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  da | Json | Elocta_clean.json
2021-06-09 18:11:04,531 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\partitionedJSONs\Elocta_clean_BILAG II.json | H | CAP |  da | Json | Elocta_clean.json
2021-06-09 18:11:04,669 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\partitionedJSONs\Elocta_clean_ ETIKETTERING .json | H | CAP |  da | Json | Elocta_clean.json
2021-06-09 18:11:04,694 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\partitionedJSONs\Elocta_clean_ INDLÆGSSEDDEL.json | H

...

2021-06-09 18:11:30,558 : List Bundle Creation Logger_1_H : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  da | 1 | Elocta_clean_BILAG II.json
2021-06-09 18:11:31,350 : List Bundle Creation Logger_1_H : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  da | 1 | Elocta_clean_BILAG II.json
2021-06-09 18:11:32,152 : List Bundle Creation Logger_1_H : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  da | 1 | Elocta_clean_BILAG II.json
2021-06-09 18:11:32,947 : List Bundle Creation Logger_1_H : Getting list bundle for MAN EU/1/15/1046/005  | H | CAP |  da | 1 | Elocta_clean_BILAG II.json
2021-06-09 18:11:33,741 : List Bundle Creation Logger_1_H : Getting list bundle for MAN EU/1/15/1046/006  | H | CAP |  da | 1 | Elocta_clean_BILAG II.json
2021-06-09 18:11:34,538 : List Bundle Creation Logger_1_H : Getting list bundle for MAN EU/1/15/1046/007  | H | CAP |  da | 1 | Elocta_clean_BILAG II.json
2021-06-09 18:11:35,298 : List Bundle Creation Logger_1_H : Getting li

Updating


2021-06-09 18:11:37,730 : List Bundle Creation Logger_1_H : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  da | 1 | Elocta_clean_BILAG II.json
2021-06-09 18:11:37,732 : Flow Logger HTML_y : Completed list bundle update/addition | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:37,734 : Flow Logger HTML_y : 1: Update/Add List Bundle,0.133 Min,0.309289 MB,0.443034 MB,48.5%
 | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:37,737 : Flow Logger HTML_y : 



||||||||||||||||||||||||||||||||2 ||||| Elocta_clean_ ETIKETTERING .json||||||||||||||||||||||||||||||||



 | H | CAP |  da | HTML | Elocta_clean.htm
2021-06-09 18:11:37,743 : Heading Extraction Elocta_clean_ ETIKETTERING .json_O : Starting Heading Extraction | H | CAP |  da | 2 | Elocta_clean_ ETIKETTERING .json


Metrics : 1: Update/Add List Bundle,0.133 Min,0.309289 MB,0.443034 MB,48.5%

Starting Heading Extraction For File :- Elocta_clean_ ETIKETTERING .json
File being processed: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\da\2021-06-09T09-34-52Z\partitionedJSONs\Elocta_clean_ ETIKETTERING .json
--------------------------------------------
ETIKETTERING


ValueError: Wrong number of items passed 12, placement implies 1

In [9]:
inputList = ['Elocta~H~CAP~de~2021-06-09T09-36-33Z.zip']
runAll(inputList)


2021-06-09 18:12:27,635 : Flow Logger HTML_A : Starting HTML Conversion To Json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:12:27,646 : Style Dictionary_w : Creating default style dictionary in file: rule_dictionary_de.json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:12:27,724 : Style Dictionary_w : Qrd Section Keys Retrieved For Style Dictionary: ZUSAMMENFASSUNG DER MERKMALE DES ARZNEIMITTELS, ANHANG II, A. ETIKETTIERUNG , B. PACKUNGSBEILAGE | H | CAP |  de | HTML | Elocta_clean.htm


Elocta~H~CAP~de~2021-06-09T09-36-33Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'de', '2021-06-09T09-36-33Z'] Elocta_clean.htm
2021-06-09T09-36-33Z de Elocta CAP H
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.json


2021-06-09 18:12:29,306 : Parser_I : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.txt | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:12:35,178 : Parser_I : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:12:35,727 : Flow Logger HTML_A : Completed HTML Conversion To Json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:12:35,729 : Flow Logger HTML_A : HTML Conversion To Json,0.1349 Min,11.443027 MB,22.149377 MB,49.1%
 | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:12:35,749 : Flow Logger HTML_A : Starting Json Split | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:12:35,755 : Style Dictionary_t : Reading style dictionary in file: rule_dictionary_de.json | H | CAP |  de | Json | Elocta_

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1349 Min,11.443027 MB,22.149377 MB,49.1%

PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.json


2021-06-09 18:12:36,195 : Partition_F : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  de | Json | Elocta_clean.json
2021-06-09 18:12:36,338 : Partition_F : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\partitionedJSONs\Elocta_clean_ANHANG II.json | H | CAP |  de | Json | Elocta_clean.json
2021-06-09 18:12:36,481 : Partition_F : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\partitionedJSONs\Elocta_clean_ ETIKETTIERUNG .json | H | CAP |  de | Json | Elocta_clean.json
2021-06-09 18:12:36,506 : Partition_F : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\partitionedJSONs\Elocta_clean_ PACKUNGSBEILAGE.json

...

POST sucessful: XML added with id 76330e0a-c40d-4e2d-9fac-f955e36f19bc
Metrics : 3: Submit FHIR Msg,0.0566 Min,0.193501 MB,2.879907 MB,49.4%

Created XML File For :- Elocta_clean_ PACKUNGSBEILAGE.json

listRegulatedAuthCodesAccrossePI ['EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008', 'EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008']


2021-06-09 18:14:09,530 : List Bundle Creation Logger_3_U : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-09 18:14:10,397 : List Bundle Creation Logger_3_U : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-09 18:14:11,501 : List Bundle Creation Logger_3_U : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-09 18:14:12,373 : List Bundle Creation Logger_3_U : Getting list bundle for MAN EU/1/15/1046/005  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-09 18:14:13,254 : List Bundle Creation Logger_3_U : Getting list bundle for MAN EU/1/15/1046/006  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-09 18:14:14,075 : List Bundle Creation Logger_3_U : Getting list bundle for MAN EU/1/15/1046/007  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-09 18:14:14,89

Updating


2021-06-09 18:14:25,993 : List Bundle Creation Logger_3_U : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-09 18:14:25,998 : Flow Logger HTML_A : Completed list bundle update/addition | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:14:26,001 : Flow Logger HTML_A : 3: Update/Add List Bundle,0.289 Min,0.338615 MB,0.909377 MB,49.7%
 | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:14:26,005 : Flow Logger HTML_A : Completed Processing Partitioned Jsons | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:14:26,009 : Flow Logger HTML_A : 3: Completed,0.0001 Min,0.14903 MB,0.151265 MB,49.7%
 | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-09 18:14:26,013 : Flow Logger HTML_A : Final Metrics,1.971 Min,0.0 MB,22.149377 MB,49.7%
 | H | CAP |  de | HTML | Elocta_clean.htm


Metrics : 3: Update/Add List Bundle,0.289 Min,0.338615 MB,0.909377 MB,49.7%

Metrics : 3: Completed,0.0001 Min,0.14903 MB,0.151265 MB,49.7%

Metrics : Final Metrics,1.971 Min,0.0 MB,22.149377 MB,49.7%



In [8]:
inputList = ['Elocta~H~CAP~el~2021-06-07T06-03-45Z.zip']
runAll(inputList)


2021-06-09 18:34:40,190 : Flow Logger HTML_8 : Starting HTML Conversion To Json | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:34:40,203 : Style Dictionary_R : Reading style dictionary in file: rule_dictionary_el.json | H | CAP |  el | HTML | Elocta_clean.htm


Elocta~H~CAP~el~2021-06-07T06-03-45Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'el', '2021-06-07T06-03-45Z'] Elocta_clean.htm
2021-06-07T06-03-45Z el Elocta CAP H


2021-06-09 18:34:40,315 : Style Dictionary_R : Qrd Section Keys Retrieved For Style Dictionary: ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ, ΠΑΡΑΡΤΗΜΑ ΙΙ, A. ΕΠΙΣΗΜΑΝΣΗ, Β. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ | H | CAP |  el | HTML | Elocta_clean.htm


Β. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.json


2021-06-09 18:34:43,335 : Parser_l : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.txt | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:34:49,913 : Parser_l : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.json | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:34:50,641 : Flow Logger HTML_8 : Completed HTML Conversion To Json | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:34:50,644 : Flow Logger HTML_8 : HTML Conversion To Json,0.1743 Min,7.495366 MB,23.644593 MB,49.7%
 | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:34:50,662 : Flow Logger HTML_8 : Starting Json Split | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:34:50,676 : Style Dictionary_d : Reading style dictionary in file: rule_dictionary_el.json | H | CAP |  el | Json | Elocta_c

...

POST sucessful: XML added with id 43190aca-5166-49ac-9e73-42ec6191eaed
Metrics : 2: Submit FHIR Msg,0.0815 Min,0.210766 MB,4.562414 MB,50.6%

Created XML File For :- Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json

listRegulatedAuthCodesAccrossePI ['EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008', 'EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008']


2021-06-09 18:36:08,325 : List Bundle Creation Logger_2_q : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  el | 2 | Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json
2021-06-09 18:36:09,256 : List Bundle Creation Logger_2_q : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  el | 2 | Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json
2021-06-09 18:36:10,082 : List Bundle Creation Logger_2_q : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  el | 2 | Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json
2021-06-09 18:36:10,901 : List Bundle Creation Logger_2_q : Getting list bundle for MAN EU/1/15/1046/005  | H | CAP |  el | 2 | Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json
2021-06-09 18:36:11,686 : List Bundle Creation Logger_2_q : Getting list bundle for MAN EU/1/15/1046/006  | H | CAP |  el | 2 | Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json
2021-06-09 18:36:12,798 : List Bundle Creation Logger_2_q : Getting list bundle for MAN EU/1/15/1046/007  | H | CAP |  el | 2 | Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json
2021-06-09 18:36:13,627 : List Bundle Creation Logge

Updating


2021-06-09 18:36:23,803 : List Bundle Creation Logger_2_q : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  el | 2 | Elocta_clean_ ΕΠΙΣΗΜΑΝΣΗ.json
2021-06-09 18:36:23,805 : Flow Logger HTML_8 : Completed list bundle update/addition | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:36:23,808 : Flow Logger HTML_8 : 2: Update/Add List Bundle,0.2718 Min,0.348878 MB,0.854079 MB,51.3%
 | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:36:23,811 : Flow Logger HTML_8 : Completed Processing Partitioned Jsons | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:36:23,814 : Flow Logger HTML_8 : 2: Completed,0.0 Min,0.149088 MB,0.151427 MB,51.3%
 | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-09 18:36:23,816 : Flow Logger HTML_8 : Final Metrics,1.7255 Min,0.0 MB,31.606994 MB,51.3%
 | H | CAP |  el | HTML | Elocta_clean.htm


Metrics : 2: Update/Add List Bundle,0.2718 Min,0.348878 MB,0.854079 MB,51.3%

Metrics : 2: Completed,0.0 Min,0.149088 MB,0.151427 MB,51.3%

Metrics : Final Metrics,1.7255 Min,0.0 MB,31.606994 MB,51.3%



In [9]:
inputList = ['Elocta~H~CAP~en~2021-06-09T12-44-02Z.zip']
runAll(inputList)

2021-06-09 18:36:29,664 : Flow Logger HTML_k : Starting HTML Conversion To Json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:29,678 : Style Dictionary_r : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:29,678 : Style Dictionary_r : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | Elocta_clean.htm


Elocta~H~CAP~en~2021-06-09T12-44-02Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\en\2021-06-09T12-44-02Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'en', '2021-06-09T12-44-02Z'] Elocta_clean.htm
2021-06-09T12-44-02Z en Elocta CAP H
B. PACKAGE LEAFLET

2021-06-09 18:36:29,799 : Style Dictionary_r : Qrd Section Keys Retrieved For Style Dictionary: SUMMARY OF PRODUCT CHARACTERISTICS, ANNEX II, A. LABELLING , B. PACKAGE LEAFLET | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:29,799 : Style Dictionary_r : Qrd Section Keys Retrieved For Style Dictionary: SUMMARY OF PRODUCT CHARACTERISTICS, ANNEX II, A. LABELLING , B. PACKAGE LEAFLET | H | CAP |  en | HTML | Elocta_clean.htm



------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\en\2021-06-09T12-44-02Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.json


2021-06-09 18:36:31,371 : Parser_1 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.txt | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:37,414 : Parser_1 : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:38,032 : Flow Logger HTML_k : Completed HTML Conversion To Json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:38,033 : Flow Logger HTML_k : HTML Conversion To Json,0.1395 Min,6.502758 MB,16.01889 MB,50.3%
 | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:38,046 : Flow Logger HTML_k : Starting Json Split | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:36:38,053 : Style Dictionary_c : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | Json | Elocta_cl

...

POST sucessful: XML added with id c2739d1d-45b7-4980-a089-ada47ddd8977
Metrics : 3: Submit FHIR Msg,0.0502 Min,0.203919 MB,2.364707 MB,50.2%

Created XML File For :- Elocta_clean_ PACKAGE LEAFLET.json

listRegulatedAuthCodesAccrossePI ['EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008', 'EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008']


2021-06-09 18:38:04,102 : List Bundle Creation Logger_3_z : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:04,102 : List Bundle Creation Logger_3_z : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:04,868 : List Bundle Creation Logger_3_z : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:04,868 : List Bundle Creation Logger_3_z : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:05,859 : List Bundle Creation Logger_3_z : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:05,859 : List Bundle Creation Logger_3_z : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:06,67

Updating


2021-06-09 18:38:18,096 : List Bundle Creation Logger_3_z : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:18,096 : List Bundle Creation Logger_3_z : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  en | 3 | Elocta_clean_ PACKAGE LEAFLET.json
2021-06-09 18:38:18,101 : Flow Logger HTML_k : Completed list bundle update/addition | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:38:18,104 : Flow Logger HTML_k : 3: Update/Add List Bundle,0.2481 Min,0.402182 MB,1.010794 MB,50.2%
 | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:38:18,106 : Flow Logger HTML_k : Completed Processing Partitioned Jsons | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:38:18,107 : Flow Logger HTML_k : 3: Completed,0.0 Min,0.003349 MB,0.152317 MB,50.2%
 | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-09 18:38:18,110 : Flow Logger HTML_k : Final Metrics,1.8058 M

Metrics : 3: Update/Add List Bundle,0.2481 Min,0.402182 MB,1.010794 MB,50.2%

Metrics : 3: Completed,0.0 Min,0.003349 MB,0.152317 MB,50.2%

Metrics : Final Metrics,1.8058 Min,0.0 MB,17.500722 MB,51.0%



In [10]:
inputList = ['Elocta~H~CAP~es~2021-06-09T09-40-41Z.zip']
runAll(inputList)


2021-06-09 18:41:03,137 : Flow Logger HTML_a : Starting HTML Conversion To Json | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:41:03,149 : Style Dictionary_A : Creating default style dictionary in file: rule_dictionary_es.json | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:41:03,248 : Style Dictionary_A : Qrd Section Keys Retrieved For Style Dictionary: FICHA TÉCNICA O RESUMEN DE LAS CARACTERÍSTICAS DEL PRODUCTO, ANEXO II, A. ETIQUETADO, B. PROSPECTO | H | CAP |  es | HTML | Elocta_clean.htm


Elocta~H~CAP~es~2021-06-09T09-40-41Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'es', '2021-06-09T09-40-41Z'] Elocta_clean.htm
2021-06-09T09-40-41Z es Elocta CAP H
B. PROSPECTO
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.json


2021-06-09 18:41:04,691 : Parser_1 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.txt | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:41:04,691 : Parser_1 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.txt | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:41:10,339 : Parser_1 : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.json | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:41:10,339 : Parser_1 : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.json | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:41:10,894 : Flow Logger HTML_a : Compl

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1294 Min,6.776444 MB,16.403882 MB,49.2%

B. PROSPECTO
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\outputJSON\Elocta_clean.json


2021-06-09 18:41:11,370 : Partition_I : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  es | Json | Elocta_clean.json
2021-06-09 18:41:11,508 : Partition_I : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\partitionedJSONs\Elocta_clean_ANEXO II.json | H | CAP |  es | Json | Elocta_clean.json
2021-06-09 18:41:11,656 : Partition_I : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\partitionedJSONs\Elocta_clean_ ETIQUETADO.json | H | CAP |  es | Json | Elocta_clean.json
2021-06-09 18:41:11,678 : Partition_I : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\es\2021-06-09T09-40-41Z\partitionedJSONs\Elocta_clean_ PROSPECTO.json | H | CAP 

...

POST sucessful: XML added with id 48fc8748-6a2c-4b58-ad20-290403198835
Metrics : 3: Submit FHIR Msg,0.0585 Min,0.203493 MB,2.387779 MB,49.3%

Created XML File For :- Elocta_clean_ PROSPECTO.json

listRegulatedAuthCodesAccrossePI ['EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008', 'EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008']


2021-06-09 18:42:41,470 : List Bundle Creation Logger_3_e : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  es | 3 | Elocta_clean_ PROSPECTO.json
2021-06-09 18:42:42,329 : List Bundle Creation Logger_3_e : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  es | 3 | Elocta_clean_ PROSPECTO.json
2021-06-09 18:42:43,139 : List Bundle Creation Logger_3_e : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  es | 3 | Elocta_clean_ PROSPECTO.json
2021-06-09 18:42:44,109 : List Bundle Creation Logger_3_e : Getting list bundle for MAN EU/1/15/1046/005  | H | CAP |  es | 3 | Elocta_clean_ PROSPECTO.json
2021-06-09 18:42:45,140 : List Bundle Creation Logger_3_e : Getting list bundle for MAN EU/1/15/1046/006  | H | CAP |  es | 3 | Elocta_clean_ PROSPECTO.json
2021-06-09 18:42:46,355 : List Bundle Creation Logger_3_e : Getting list bundle for MAN EU/1/15/1046/007  | H | CAP |  es | 3 | Elocta_clean_ PROSPECTO.json
2021-06-09 18:42:48,228 : List Bundle Creation Logger_3_e 

Updating


2021-06-09 18:42:59,748 : List Bundle Creation Logger_3_e : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  es | 3 | Elocta_clean_ PROSPECTO.json
2021-06-09 18:42:59,750 : Flow Logger HTML_a : Completed list bundle update/addition | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:42:59,752 : Flow Logger HTML_a : 3: Update/Add List Bundle,0.3277 Min,0.38222 MB,1.142493 MB,49.2%
 | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:42:59,755 : Flow Logger HTML_a : Completed Processing Partitioned Jsons | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:42:59,756 : Flow Logger HTML_a : 3: Completed,0.0 Min,0.003349 MB,0.152317 MB,49.2%
 | H | CAP |  es | HTML | Elocta_clean.htm
2021-06-09 18:42:59,759 : Flow Logger HTML_a : Final Metrics,1.942 Min,0.0 MB,17.908566 MB,49.4%
 | H | CAP |  es | HTML | Elocta_clean.htm


Metrics : 3: Update/Add List Bundle,0.3277 Min,0.38222 MB,1.142493 MB,49.2%

Metrics : 3: Completed,0.0 Min,0.003349 MB,0.152317 MB,49.2%

Metrics : Final Metrics,1.942 Min,0.0 MB,17.908566 MB,49.4%



In [11]:
inputList = ['Elocta~H~CAP~et~2021-06-09T09-44-12Z.zip']
runAll(inputList)


2021-06-09 18:43:14,215 : Flow Logger HTML_R : Starting HTML Conversion To Json | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:14,228 : Style Dictionary_F : Creating default style dictionary in file: rule_dictionary_et.json | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:14,228 : Style Dictionary_F : Creating default style dictionary in file: rule_dictionary_et.json | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:14,309 : Style Dictionary_F : Qrd Section Keys Retrieved For Style Dictionary: RAVIMI OMADUSTE KOKKUVÕTE, II LISA, A. PAKENDI MÄRGISTUS, B. PAKENDI INFOLEHT | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:14,309 : Style Dictionary_F : Qrd Section Keys Retrieved For Style Dictionary: RAVIMI OMADUSTE KOKKUVÕTE, II LISA, A. PAKENDI MÄRGISTUS, B. PAKENDI INFOLEHT | H | CAP |  et | HTML | Elocta_clean.htm


Elocta~H~CAP~et~2021-06-09T09-44-12Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'et', '2021-06-09T09-44-12Z'] Elocta_clean.htm
2021-06-09T09-44-12Z et Elocta CAP H
B. PAKENDI INFOLEHT
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\outputJSON\Elocta_clean.json


2021-06-09 18:43:15,705 : Parser_a : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\outputJSON\Elocta_clean.txt | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:20,987 : Parser_a : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\outputJSON\Elocta_clean.json | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:21,510 : Flow Logger HTML_R : Completed HTML Conversion To Json | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:21,511 : Flow Logger HTML_R : HTML Conversion To Json,0.1216 Min,6.430332 MB,16.264251 MB,49.4%
 | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:21,523 : Flow Logger HTML_R : Starting Json Split | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-09 18:43:21,530 : Style Dictionary_m : Reading style dictionary in file: rule_dictionary_et.json | H | CAP |  et | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1216 Min,6.430332 MB,16.264251 MB,49.4%

B. PAKENDI INFOLEHT
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\outputJSON\Elocta_clean.json


2021-06-09 18:43:21,990 : Partition_0 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  et | Json | Elocta_clean.json
2021-06-09 18:43:22,134 : Partition_0 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\partitionedJSONs\Elocta_clean_II LISA.json | H | CAP |  et | Json | Elocta_clean.json
2021-06-09 18:43:22,280 : Partition_0 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\partitionedJSONs\Elocta_clean_ PAKENDI MÄRGISTUS.json | H | CAP |  et | Json | Elocta_clean.json
2021-06-09 18:43:22,305 : Partition_0 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\et\2021-06-09T09-44-12Z\partitionedJSONs\Elocta_clean_ PAKENDI INFOLEHT.js

Metrics : Split Json,0.0135 Min,0.372101 MB,17.884493 MB,49.4%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'et'

In [12]:
inputList = ['Elocta~H~CAP~fi~2021-06-09T09-51-23Z.zip']
runAll(inputList)


2021-06-09 18:43:37,312 : Flow Logger HTML_S : Starting HTML Conversion To Json | H | CAP |  fi | HTML | Elocta_clean.htm
2021-06-09 18:43:37,324 : Style Dictionary_9 : Creating default style dictionary in file: rule_dictionary_fi.json | H | CAP |  fi | HTML | Elocta_clean.htm
2021-06-09 18:43:37,406 : Style Dictionary_9 : Qrd Section Keys Retrieved For Style Dictionary: VALMISTEYHTEENVETO, LIITE II , A. MYYNTIPÄÄLLYSMERKINNÄT, B. PAKKAUSSELOSTE | H | CAP |  fi | HTML | Elocta_clean.htm


Elocta~H~CAP~fi~2021-06-09T09-51-23Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'fi', '2021-06-09T09-51-23Z'] Elocta_clean.htm
2021-06-09T09-51-23Z fi Elocta CAP H
B. PAKKAUSSELOSTE
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\outputJSON\Elocta_clean.json


2021-06-09 18:43:38,911 : Parser_s : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\outputJSON\Elocta_clean.txt | H | CAP |  fi | HTML | Elocta_clean.htm
2021-06-09 18:43:44,663 : Parser_s : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\outputJSON\Elocta_clean.json | H | CAP |  fi | HTML | Elocta_clean.htm
2021-06-09 18:43:45,183 : Flow Logger HTML_S : Completed HTML Conversion To Json | H | CAP |  fi | HTML | Elocta_clean.htm
2021-06-09 18:43:45,186 : Flow Logger HTML_S : HTML Conversion To Json,0.1313 Min,8.376023 MB,18.247927 MB,49.4%
 | H | CAP |  fi | HTML | Elocta_clean.htm
2021-06-09 18:43:45,202 : Flow Logger HTML_S : Starting Json Split | H | CAP |  fi | HTML | Elocta_clean.htm
2021-06-09 18:43:45,209 : Style Dictionary_B : Reading style dictionary in file: rule_dictionary_fi.json | H | CAP |  fi | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1313 Min,8.376023 MB,18.247927 MB,49.4%

B. PAKKAUSSELOSTE
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\outputJSON\Elocta_clean.json


2021-06-09 18:43:45,676 : Partition_v : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  fi | Json | Elocta_clean.json
2021-06-09 18:43:45,832 : Partition_v : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\partitionedJSONs\Elocta_clean_LIITE II .json | H | CAP |  fi | Json | Elocta_clean.json
2021-06-09 18:43:46,011 : Partition_v : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\partitionedJSONs\Elocta_clean_ MYYNTIPÄÄLLYSMERKINNÄT.json | H | CAP |  fi | Json | Elocta_clean.json
2021-06-09 18:43:46,052 : Partition_v : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fi\2021-06-09T09-51-23Z\partitionedJSONs\Elocta_clean_ PAKKAUSSELOS

Metrics : Split Json,0.0148 Min,0.347068 MB,18.054118 MB,49.4%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'fi'

In [13]:
inputList = ['Elocta~H~CAP~fr~2021-06-09T09-52-44Z.zip']
runAll(inputList)


2021-06-09 18:44:12,243 : Flow Logger HTML_8 : Starting HTML Conversion To Json | H | CAP |  fr | HTML | Elocta_clean.htm
2021-06-09 18:44:12,243 : Flow Logger HTML_8 : Starting HTML Conversion To Json | H | CAP |  fr | HTML | Elocta_clean.htm
2021-06-09 18:44:12,258 : Style Dictionary_e : Creating default style dictionary in file: rule_dictionary_fr.json | H | CAP |  fr | HTML | Elocta_clean.htm


Elocta~H~CAP~fr~2021-06-09T09-52-44Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'fr', '2021-06-09T09-52-44Z'] Elocta_clean.htm
2021-06-09T09-52-44Z fr Elocta CAP H
B. NOTICE

2021-06-09 18:44:12,359 : Style Dictionary_e : Qrd Section Keys Retrieved For Style Dictionary: RÉSUMÉ DES CARACTÉRISTIQUES DU PRODUIT, ANNEXE II, A. ÉTIQUETAGE, B. NOTICE | H | CAP |  fr | HTML | Elocta_clean.htm



------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\outputJSON\Elocta_clean.json


2021-06-09 18:44:13,743 : Parser_E : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\outputJSON\Elocta_clean.txt | H | CAP |  fr | HTML | Elocta_clean.htm
2021-06-09 18:44:19,032 : Parser_E : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\outputJSON\Elocta_clean.json | H | CAP |  fr | HTML | Elocta_clean.htm
2021-06-09 18:44:19,570 : Flow Logger HTML_8 : Completed HTML Conversion To Json | H | CAP |  fr | HTML | Elocta_clean.htm
2021-06-09 18:44:19,570 : Flow Logger HTML_8 : Completed HTML Conversion To Json | H | CAP |  fr | HTML | Elocta_clean.htm
2021-06-09 18:44:19,575 : Flow Logger HTML_8 : HTML Conversion To Json,0.1222 Min,6.712312 MB,16.454664 MB,49.4%
 | H | CAP |  fr | HTML | Elocta_clean.htm
2021-06-09 18:44:19,575 : Flow Logger HTML_8 : HTML Conversion To Json,0.1222 Min,6.712312 MB,16.454664 MB,49.4%
 | H | CAP 

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1222 Min,6.712312 MB,16.454664 MB,49.4%

B. NOTICE
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\outputJSON\Elocta_clean.json


2021-06-09 18:44:20,082 : Partition_a : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  fr | Json | Elocta_clean.json
2021-06-09 18:44:20,248 : Partition_a : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\partitionedJSONs\Elocta_clean_ANNEXE II.json | H | CAP |  fr | Json | Elocta_clean.json
2021-06-09 18:44:20,417 : Partition_a : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\partitionedJSONs\Elocta_clean_ ÉTIQUETAGE.json | H | CAP |  fr | Json | Elocta_clean.json
2021-06-09 18:44:20,442 : Partition_a : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\fr\2021-06-09T09-52-44Z\partitionedJSONs\Elocta_clean_ NOTICE.json | H | CAP | 

Metrics : Split Json,0.0146 Min,0.202588 MB,18.039423 MB,49.4%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'fr'

In [15]:
inputList = ['Elocta~H~CAP~hr~2021-06-09T09-54-07Z.zip']
runAll(inputList)

2021-06-09 18:45:35,620 : Flow Logger HTML_b : Starting HTML Conversion To Json | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:35,634 : Style Dictionary_g : Reading style dictionary in file: rule_dictionary_hr.json | H | CAP |  hr | HTML | Elocta_clean.htm


Elocta~H~CAP~hr~2021-06-09T09-54-07Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'hr', '2021-06-09T09-54-07Z'] Elocta_clean.htm
2021-06-09T09-54-07Z hr Elocta CAP H
B. UPUTA O LIJEKU

2021-06-09 18:45:35,721 : Style Dictionary_g : Qrd Section Keys Retrieved For Style Dictionary: SAŽETAK OPISA SVOJSTAVA LIJEKA, PRILOG II, A. OZNAČIVANJE, B. UPUTA O LIJEKU | H | CAP |  hr | HTML | Elocta_clean.htm



------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\outputJSON\Elocta_clean.json


2021-06-09 18:45:37,108 : Parser_A : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\outputJSON\Elocta_clean.txt | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:43,599 : Parser_A : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\outputJSON\Elocta_clean.json | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,172 : Flow Logger HTML_b : Completed HTML Conversion To Json | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,174 : Flow Logger HTML_b : HTML Conversion To Json,0.1426 Min,7.018706 MB,17.231053 MB,43.6%
 | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,186 : Flow Logger HTML_b : Starting Json Split | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,191 : Style Dictionary_e : Reading style dictionary in file: rule_dictionary_hr.json | H | CAP |  hr | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1426 Min,7.018706 MB,17.231053 MB,43.6%

B. UPUTA O LIJEKU
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\outputJSON\Elocta_clean.json


2021-06-09 18:45:44,715 : Partition_f : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hr\2021-06-09T09-54-07Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  hr | Json | Elocta_clean.json
2021-06-09 18:45:44,805 : Flow Logger HTML_b : ['Elocta_clean_SmPC.json'] | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,807 : Flow Logger HTML_b : Completed Json Split | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,808 : Flow Logger HTML_b : Split Json,0.0104 Min,0.358037 MB,20.106785 MB,43.6%
 | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,812 : Flow Logger HTML_b : Started Processing Partitioned Jsons | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,814 : Flow Logger HTML_b : 



||||||||||||||||||||||||||||||||0 ||||| Elocta_clean_SmPC.json||||||||||||||||||||||||||||||||



 | H | CAP |  hr | HTML | Elocta_clean.htm
2021-06-09 18:45:44,820 : Heading Extraction Elocta_clean_S

Metrics : Split Json,0.0104 Min,0.358037 MB,20.106785 MB,43.6%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'hr'

In [16]:
inputList = ['Elocta~H~CAP~hu~2021-06-09T09-56-52Z.zip']
runAll(inputList)

2021-06-09 18:48:27,703 : Flow Logger HTML_M : Starting HTML Conversion To Json | H | CAP |  hu | HTML | Elocta_clean.htm
2021-06-09 18:48:27,714 : Style Dictionary_E : Reading style dictionary in file: rule_dictionary_hu.json | H | CAP |  hu | HTML | Elocta_clean.htm


Elocta~H~CAP~hu~2021-06-09T09-56-52Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'hu', '2021-06-09T09-56-52Z'] Elocta_clean.htm
2021-06-09T09-56-52Z hu Elocta CAP H
B. BETEGTÁJÉKOZTATÓ

2021-06-09 18:48:27,810 : Style Dictionary_E : Qrd Section Keys Retrieved For Style Dictionary: ALKALMAZÁSI ELŐÍRÁS, II. MELLÉKLET, A. CÍMKESZÖVEG, B. BETEGTÁJÉKOZTATÓ | H | CAP |  hu | HTML | Elocta_clean.htm



------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\outputJSON\Elocta_clean.json


2021-06-09 18:48:29,307 : Parser_u : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\outputJSON\Elocta_clean.txt | H | CAP |  hu | HTML | Elocta_clean.htm
2021-06-09 18:48:35,425 : Parser_u : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\outputJSON\Elocta_clean.json | H | CAP |  hu | HTML | Elocta_clean.htm
2021-06-09 18:48:36,089 : Flow Logger HTML_M : Completed HTML Conversion To Json | H | CAP |  hu | HTML | Elocta_clean.htm
2021-06-09 18:48:36,092 : Flow Logger HTML_M : HTML Conversion To Json,0.1398 Min,6.640518 MB,15.503134 MB,43.2%
 | H | CAP |  hu | HTML | Elocta_clean.htm
2021-06-09 18:48:36,109 : Flow Logger HTML_M : Starting Json Split | H | CAP |  hu | HTML | Elocta_clean.htm
2021-06-09 18:48:36,118 : Style Dictionary_3 : Reading style dictionary in file: rule_dictionary_hu.json | H | CAP |  hu | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1398 Min,6.640518 MB,15.503134 MB,43.2%

B. BETEGTÁJÉKOZTATÓ
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\outputJSON\Elocta_clean.json


2021-06-09 18:48:36,614 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  hu | Json | Elocta_clean.json
2021-06-09 18:48:36,784 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\partitionedJSONs\Elocta_clean_II. MELLÉKLET.json | H | CAP |  hu | Json | Elocta_clean.json
2021-06-09 18:48:36,937 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\partitionedJSONs\Elocta_clean_ CÍMKESZÖVEG.json | H | CAP |  hu | Json | Elocta_clean.json
2021-06-09 18:48:36,965 : Partition_K : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\hu\2021-06-09T09-56-52Z\partitionedJSONs\Elocta_clean_ BETEGTÁJÉKOZTATÓ.js

Metrics : Split Json,0.0147 Min,0.365348 MB,17.417275 MB,43.2%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'hu'

In [17]:
inputList = ['Elocta~H~CAP~is~2021-06-09T09-57-48Z.zip']
runAll(inputList)

2021-06-09 18:48:50,106 : Flow Logger HTML_z : Starting HTML Conversion To Json | H | CAP |  is | HTML | Elocta_clean.htm
2021-06-09 18:48:50,118 : Style Dictionary_s : Creating default style dictionary in file: rule_dictionary_is.json | H | CAP |  is | HTML | Elocta_clean.htm


Elocta~H~CAP~is~2021-06-09T09-57-48Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\is\2021-06-09T09-57-48Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'is', '2021-06-09T09-57-48Z'] Elocta_clean.htm
2021-06-09T09-57-48Z is Elocta CAP H


2021-06-09 18:48:50,200 : Style Dictionary_s : Language Code Not Found in QRD Template | H | CAP |  is | HTML | Elocta_clean.htm
Traceback (most recent call last):
  File "C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\code\parse\rulebook\rulebook.py", line 108, in getSectionKeys
    raise LanguageErrorQrdTemplate("Language not found in QRD template")
parse.rulebook.rulebook.LanguageErrorQrdTemplate: Language not found in QRD template
2021-06-09 18:48:50,222 : Style Dictionary_s : Language Code Not Found in QRD Template | H | CAP |  is | HTML | Elocta_clean.htm


LanguageErrorQrdTemplate: Language not found in QRD template

In [19]:
inputList = ['Elocta~H~CAP~lt~2021-06-09T10-00-51Z.zip']
runAll(inputList)

2021-06-09 18:49:28,496 : Flow Logger HTML_T : Starting HTML Conversion To Json | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:28,496 : Flow Logger HTML_T : Starting HTML Conversion To Json | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:28,511 : Style Dictionary_n : Reading style dictionary in file: rule_dictionary_lt.json | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:28,612 : Style Dictionary_n : Qrd Section Keys Retrieved For Style Dictionary: PREPARATO CHARAKTERISTIKŲ SANTRAUKA, II PRIEDAS, A. ŽENKLINIMAS, B. PAKUOTĖS LAPELIS | H | CAP |  lt | HTML | Elocta_clean.htm


Elocta~H~CAP~lt~2021-06-09T10-00-51Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'lt', '2021-06-09T10-00-51Z'] Elocta_clean.htm
2021-06-09T10-00-51Z lt Elocta CAP H
B. PAKUOTĖS LAPELIS
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\outputJSON\Elocta_clean.json


2021-06-09 18:49:29,945 : Parser_f : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\outputJSON\Elocta_clean.txt | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:35,461 : Parser_f : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\outputJSON\Elocta_clean.json | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:35,963 : Flow Logger HTML_T : Completed HTML Conversion To Json | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:35,963 : Flow Logger HTML_T : Completed HTML Conversion To Json | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:35,967 : Flow Logger HTML_T : HTML Conversion To Json,0.1245 Min,6.668906 MB,16.333742 MB,43.4%
 | H | CAP |  lt | HTML | Elocta_clean.htm
2021-06-09 18:49:35,967 : Flow Logger HTML_T : HTML Conversion To Json,0.1245 Min,6.668906 MB,16.333742 MB,43.4%
 | H | CAP 

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1245 Min,6.668906 MB,16.333742 MB,43.4%

B. PAKUOTĖS LAPELIS
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\outputJSON\Elocta_clean.json


2021-06-09 18:49:36,448 : Partition_d : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  lt | Json | Elocta_clean.json
2021-06-09 18:49:36,448 : Partition_d : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  lt | Json | Elocta_clean.json
2021-06-09 18:49:36,597 : Partition_d : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\partitionedJSONs\Elocta_clean_II PRIEDAS.json | H | CAP |  lt | Json | Elocta_clean.json
2021-06-09 18:49:36,597 : Partition_d : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lt\2021-06-09T10-00-51Z\partitionedJSONs\Elocta_clean_II PRIEDAS.json | H | CAP |  lt

Metrics : Split Json,0.0141 Min,0.341709 MB,17.859883 MB,43.4%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'lt'

In [20]:
inputList = ['Elocta~H~CAP~lv~2021-06-09T10-02-03Z.zip']
runAll(inputList)

2021-06-09 18:49:46,001 : Flow Logger HTML_p : Starting HTML Conversion To Json | H | CAP |  lv | HTML | Elocta_clean.htm
2021-06-09 18:49:46,016 : Style Dictionary_k : Creating default style dictionary in file: rule_dictionary_lv.json | H | CAP |  lv | HTML | Elocta_clean.htm


Elocta~H~CAP~lv~2021-06-09T10-02-03Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'lv', '2021-06-09T10-02-03Z'] Elocta_clean.htm
2021-06-09T10-02-03Z lv Elocta CAP H
B. LIETOŠANAS INSTRUKCIJA

2021-06-09 18:49:46,115 : Style Dictionary_k : Qrd Section Keys Retrieved For Style Dictionary: ZĀĻU APRAKSTS, II PIELIKUMS, A. MARĶĒJUMA TEKSTS, B. LIETOŠANAS INSTRUKCIJA | H | CAP |  lv | HTML | Elocta_clean.htm



------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\outputJSON\Elocta_clean.json


2021-06-09 18:49:47,492 : Parser_I : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\outputJSON\Elocta_clean.txt | H | CAP |  lv | HTML | Elocta_clean.htm
2021-06-09 18:49:53,083 : Parser_I : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\outputJSON\Elocta_clean.json | H | CAP |  lv | HTML | Elocta_clean.htm
2021-06-09 18:49:53,616 : Flow Logger HTML_p : Completed HTML Conversion To Json | H | CAP |  lv | HTML | Elocta_clean.htm
2021-06-09 18:49:53,618 : Flow Logger HTML_p : HTML Conversion To Json,0.127 Min,6.828444 MB,17.139956 MB,43.6%
 | H | CAP |  lv | HTML | Elocta_clean.htm
2021-06-09 18:49:53,631 : Flow Logger HTML_p : Starting Json Split | H | CAP |  lv | HTML | Elocta_clean.htm
2021-06-09 18:49:53,639 : Style Dictionary_u : Reading style dictionary in file: rule_dictionary_lv.json | H | CAP |  lv | Json | Elocta_cl

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.127 Min,6.828444 MB,17.139956 MB,43.6%

B. LIETOŠANAS INSTRUKCIJA
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\outputJSON\Elocta_clean.json


2021-06-09 18:49:54,096 : Partition_j : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  lv | Json | Elocta_clean.json
2021-06-09 18:49:54,228 : Partition_j : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\partitionedJSONs\Elocta_clean_II PIELIKUMS.json | H | CAP |  lv | Json | Elocta_clean.json
2021-06-09 18:49:54,380 : Partition_j : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\partitionedJSONs\Elocta_clean_ MARĶĒJUMA TEKSTS.json | H | CAP |  lv | Json | Elocta_clean.json
2021-06-09 18:49:54,408 : Partition_j : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\lv\2021-06-09T10-02-03Z\partitionedJSONs\Elocta_clean_ LIETOŠANAS INST

Metrics : Split Json,0.0135 Min,0.343389 MB,18.475624 MB,43.5%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'lv'

In [21]:
inputList = ['Elocta~H~CAP~mt~2021-06-09T10-03-45Z.zip']
runAll(inputList)

2021-06-09 18:50:02,932 : Flow Logger HTML_0 : Starting HTML Conversion To Json | H | CAP |  mt | HTML | Elocta_clean.htm
2021-06-09 18:50:02,947 : Style Dictionary_p : Creating default style dictionary in file: rule_dictionary_mt.json | H | CAP |  mt | HTML | Elocta_clean.htm
2021-06-09 18:50:02,947 : Style Dictionary_p : Creating default style dictionary in file: rule_dictionary_mt.json | H | CAP |  mt | HTML | Elocta_clean.htm
2021-06-09 18:50:03,037 : Style Dictionary_p : Qrd Section Keys Retrieved For Style Dictionary: SOMMARJU TAL-KARATTERISTIĊI TAL-PRODOTT, ANNESS II, A. TIKKETTAR, B. FULJETT TA’ TAGĦRIF | H | CAP |  mt | HTML | Elocta_clean.htm


Elocta~H~CAP~mt~2021-06-09T10-03-45Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'mt', '2021-06-09T10-03-45Z'] Elocta_clean.htm
2021-06-09T10-03-45Z mt Elocta CAP H
B. FULJETT TA’ TAGĦRIF


2021-06-09 18:50:03,037 : Style Dictionary_p : Qrd Section Keys Retrieved For Style Dictionary: SOMMARJU TAL-KARATTERISTIĊI TAL-PRODOTT, ANNESS II, A. TIKKETTAR, B. FULJETT TA’ TAGĦRIF | H | CAP |  mt | HTML | Elocta_clean.htm


------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.json


2021-06-09 18:50:04,976 : Parser_1 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.txt | H | CAP |  mt | HTML | Elocta_clean.htm
2021-06-09 18:50:04,976 : Parser_1 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.txt | H | CAP |  mt | HTML | Elocta_clean.htm
2021-06-09 18:50:04,976 : Parser_1 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.txt | H | CAP |  mt | HTML | Elocta_clean.htm
2021-06-09 18:50:11,711 : Parser_1 : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.json | H | CAP |  mt | HTML | Elocta_clean.htm
2021-06-09 18:50:11,711 : Parser_1 :

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1566 Min,7.260623 MB,17.686004 MB,43.7%

B. FULJETT TA’ TAGĦRIF
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\outputJSON\Elocta_clean.json


2021-06-09 18:50:12,811 : Partition_c : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  mt | Json | Elocta_clean.json
2021-06-09 18:50:12,948 : Partition_c : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\partitionedJSONs\Elocta_clean_ANNESS II.json | H | CAP |  mt | Json | Elocta_clean.json
2021-06-09 18:50:13,109 : Partition_c : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\partitionedJSONs\Elocta_clean_ TIKKETTAR.json | H | CAP |  mt | Json | Elocta_clean.json
2021-06-09 18:50:13,141 : Partition_c : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\mt\2021-06-09T10-03-45Z\partitionedJSONs\Elocta_clean_ FULJETT TA’ TAGĦRIF.json 

Metrics : Split Json,0.014 Min,0.127434 MB,18.490489 MB,43.6%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'mt'

In [22]:
inputList = ['Elocta~H~CAP~nl~2021-06-09T10-04-39Z.zip']
runAll(inputList)


2021-06-09 18:50:33,110 : Flow Logger HTML_f : Starting HTML Conversion To Json | H | CAP |  nl | HTML | Elocta_clean.htm
2021-06-09 18:50:33,124 : Style Dictionary_S : Creating default style dictionary in file: rule_dictionary_nl.json | H | CAP |  nl | HTML | Elocta_clean.htm
2021-06-09 18:50:33,124 : Style Dictionary_S : Creating default style dictionary in file: rule_dictionary_nl.json | H | CAP |  nl | HTML | Elocta_clean.htm
2021-06-09 18:50:33,220 : Style Dictionary_S : Qrd Section Keys Retrieved For Style Dictionary: SAMENVATTING VAN DE PRODUCTKENMERKEN, BIJLAGE II, A. ETIKETTERING, B. BIJSLUITER | H | CAP |  nl | HTML | Elocta_clean.htm


Elocta~H~CAP~nl~2021-06-09T10-04-39Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'nl', '2021-06-09T10-04-39Z'] Elocta_clean.htm
2021-06-09T10-04-39Z nl Elocta CAP H
B. BIJSLUITER


2021-06-09 18:50:33,220 : Style Dictionary_S : Qrd Section Keys Retrieved For Style Dictionary: SAMENVATTING VAN DE PRODUCTKENMERKEN, BIJLAGE II, A. ETIKETTERING, B. BIJSLUITER | H | CAP |  nl | HTML | Elocta_clean.htm


------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.json


2021-06-09 18:50:34,848 : Parser_R : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.txt | H | CAP |  nl | HTML | Elocta_clean.htm
2021-06-09 18:50:34,848 : Parser_R : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.txt | H | CAP |  nl | HTML | Elocta_clean.htm
2021-06-09 18:50:40,739 : Parser_R : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.json | H | CAP |  nl | HTML | Elocta_clean.htm
2021-06-09 18:50:40,739 : Parser_R : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.json | H | CAP |  nl | HTML | Elocta_clean.htm
2021-06-09 18:50:41,296 : Flow Logger HTML_f : Compl

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1365 Min,6.549834 MB,15.068964 MB,43.8%

B. BIJSLUITER
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\outputJSON\Elocta_clean.json


2021-06-09 18:50:41,786 : Partition_m : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  nl | Json | Elocta_clean.json
2021-06-09 18:50:41,935 : Partition_m : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\partitionedJSONs\Elocta_clean_BIJLAGE II.json | H | CAP |  nl | Json | Elocta_clean.json
2021-06-09 18:50:42,106 : Partition_m : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\partitionedJSONs\Elocta_clean_ ETIKETTERING.json | H | CAP |  nl | Json | Elocta_clean.json
2021-06-09 18:50:42,135 : Partition_m : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\nl\2021-06-09T10-04-39Z\partitionedJSONs\Elocta_clean_ BIJSLUITER.json | H |

Metrics : Split Json,0.0142 Min,0.341151 MB,16.997235 MB,43.8%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'nl'

In [24]:
inputList = ['Elocta~H~CAP~no~2021-06-09T10-07-44Z.zip']
runAll(inputList)


2021-06-09 18:52:53,721 : Flow Logger HTML_X : Starting HTML Conversion To Json | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:52:53,733 : Style Dictionary_m : Reading style dictionary in file: rule_dictionary_no.json | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:52:53,733 : Style Dictionary_m : Reading style dictionary in file: rule_dictionary_no.json | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:52:53,733 : Style Dictionary_m : Reading style dictionary in file: rule_dictionary_no.json | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:52:53,827 : Style Dictionary_m : Qrd Section Keys Retrieved For Style Dictionary: PREPARATOMTALE, VEDLEGG II, A. MERKING, B. PAKNINGSVEDLEGG | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:52:53,827 : Style Dictionary_m : Qrd Section Keys Retrieved For Style Dictionary: PREPARATOMTALE, VEDLEGG II, A. MERKING, B. PAKNINGSVEDLEGG | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:52:53,827 : Style Dictionary_

Elocta~H~CAP~no~2021-06-09T10-07-44Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'no', '2021-06-09T10-07-44Z'] Elocta_clean.htm
2021-06-09T10-07-44Z no Elocta CAP H
B. PAKNINGSVEDLEGG
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.json


2021-06-09 18:52:55,274 : Parser_l : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.txt | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:52:55,274 : Parser_l : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.txt | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:53:00,670 : Parser_l : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.json | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:53:00,670 : Parser_l : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.json | H | CAP |  no | HTML | Elocta_clean.htm
2021-06-09 18:53:01,227 : Flow Logger HTML_X : Compl

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1251 Min,6.810593 MB,16.26088 MB,43.9%

B. PAKNINGSVEDLEGG
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\outputJSON\Elocta_clean.json


2021-06-09 18:53:01,707 : Partition_J : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  no | Json | Elocta_clean.json
2021-06-09 18:53:01,840 : Partition_J : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\partitionedJSONs\Elocta_clean_VEDLEGG II.json | H | CAP |  no | Json | Elocta_clean.json
2021-06-09 18:53:01,973 : Partition_J : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\partitionedJSONs\Elocta_clean_ MERKING.json | H | CAP |  no | Json | Elocta_clean.json
2021-06-09 18:53:02,000 : Partition_J : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\no\2021-06-09T10-07-44Z\partitionedJSONs\Elocta_clean_ PAKNINGSVEDLEGG.json | H |

Metrics : Split Json,0.0131 Min,0.35015 MB,17.623482 MB,43.9%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'no'

In [26]:
inputList = ['Elocta~H~CAP~pl~2021-06-09T10-08-48Z.zip']
runAll(inputList)


2021-06-09 18:54:13,364 : Flow Logger HTML_E : Starting HTML Conversion To Json | H | CAP |  pl | HTML | Elocta_clean.htm
2021-06-09 18:54:13,374 : Style Dictionary_W : Reading style dictionary in file: rule_dictionary_pl.json | H | CAP |  pl | HTML | Elocta_clean.htm
2021-06-09 18:54:13,449 : Style Dictionary_W : Qrd Section Keys Retrieved For Style Dictionary: CHARAKTERYSTYKA PRODUKTU LECZNICZEGO, ANEKS II, A. OZNAKOWANIE OPAKOWAŃ, B. ULOTKA DLA PACJENTA | H | CAP |  pl | HTML | Elocta_clean.htm


Elocta~H~CAP~pl~2021-06-09T10-08-48Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'pl', '2021-06-09T10-08-48Z'] Elocta_clean.htm
2021-06-09T10-08-48Z pl Elocta CAP H
B. ULOTKA DLA PACJENTA
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\outputJSON\Elocta_clean.json


2021-06-09 18:54:14,862 : Parser_d : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\outputJSON\Elocta_clean.txt | H | CAP |  pl | HTML | Elocta_clean.htm
2021-06-09 18:54:20,363 : Parser_d : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\outputJSON\Elocta_clean.json | H | CAP |  pl | HTML | Elocta_clean.htm
2021-06-09 18:54:20,883 : Flow Logger HTML_E : Completed HTML Conversion To Json | H | CAP |  pl | HTML | Elocta_clean.htm
2021-06-09 18:54:20,885 : Flow Logger HTML_E : HTML Conversion To Json,0.1254 Min,6.891509 MB,17.10514 MB,43.3%
 | H | CAP |  pl | HTML | Elocta_clean.htm
2021-06-09 18:54:20,898 : Flow Logger HTML_E : Starting Json Split | H | CAP |  pl | HTML | Elocta_clean.htm
2021-06-09 18:54:20,905 : Style Dictionary_w : Reading style dictionary in file: rule_dictionary_pl.json | H | CAP |  pl | Json | Elocta_cl

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1254 Min,6.891509 MB,17.10514 MB,43.3%

B. ULOTKA DLA PACJENTA
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\outputJSON\Elocta_clean.json


2021-06-09 18:54:21,381 : Partition_E : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  pl | Json | Elocta_clean.json
2021-06-09 18:54:21,541 : Partition_E : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\partitionedJSONs\Elocta_clean_ANEKS II.json | H | CAP |  pl | Json | Elocta_clean.json
2021-06-09 18:54:21,693 : Partition_E : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\partitionedJSONs\Elocta_clean_ OZNAKOWANIE OPAKOWAŃ.json | H | CAP |  pl | Json | Elocta_clean.json
2021-06-09 18:54:21,719 : Partition_E : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pl\2021-06-09T10-08-48Z\partitionedJSONs\Elocta_clean_ ULOTKA DLA PACJ

Metrics : Split Json,0.0142 Min,0.363867 MB,18.24111 MB,43.3%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'pl'

In [27]:
inputList = ['Elocta~H~CAP~pt~2021-06-09T10-09-45Z.zip']
runAll(inputList)


2021-06-09 18:54:27,389 : Flow Logger HTML_v : Starting HTML Conversion To Json | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:27,401 : Style Dictionary_n : Creating default style dictionary in file: rule_dictionary_pt.json | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:27,401 : Style Dictionary_n : Creating default style dictionary in file: rule_dictionary_pt.json | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:27,498 : Style Dictionary_n : Qrd Section Keys Retrieved For Style Dictionary: RESUMO DAS CARACTERÍSTICAS DO MEDICAMENTO, ANEXO II, A. ROTULAGEM, B. FOLHETO INFORMATIVO | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:27,498 : Style Dictionary_n : Qrd Section Keys Retrieved For Style Dictionary: RESUMO DAS CARACTERÍSTICAS DO MEDICAMENTO, ANEXO II, A. ROTULAGEM, B. FOLHETO INFORMATIVO | H | CAP |  pt | HTML | Elocta_clean.htm


Elocta~H~CAP~pt~2021-06-09T10-09-45Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'pt', '2021-06-09T10-09-45Z'] Elocta_clean.htm
2021-06-09T10-09-45Z pt Elocta CAP H
B. FOLHETO INFORMATIVO
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.json


2021-06-09 18:54:29,057 : Parser_3 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.txt | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:29,057 : Parser_3 : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.txt | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:35,322 : Parser_3 : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.json | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:35,322 : Parser_3 : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.json | H | CAP |  pt | HTML | Elocta_clean.htm
2021-06-09 18:54:35,868 : Flow Logger HTML_v : Compl

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1415 Min,6.798016 MB,16.636652 MB,44.2%

B. FOLHETO INFORMATIVO
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\outputJSON\Elocta_clean.json


2021-06-09 18:54:36,392 : Partition_3 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  pt | Json | Elocta_clean.json
2021-06-09 18:54:36,547 : Partition_3 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\partitionedJSONs\Elocta_clean_ANEXO II.json | H | CAP |  pt | Json | Elocta_clean.json
2021-06-09 18:54:36,681 : Partition_3 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\partitionedJSONs\Elocta_clean_ ROTULAGEM.json | H | CAP |  pt | Json | Elocta_clean.json
2021-06-09 18:54:36,705 : Partition_3 : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\pt\2021-06-09T10-09-45Z\partitionedJSONs\Elocta_clean_ FOLHETO INFORMATIVO.json |

Metrics : Split Json,0.0141 Min,0.368132 MB,18.060003 MB,44.2%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'pt'

In [29]:
inputList = ['Elocta~H~CAP~ro~2021-06-09T10-12-05Z.zip']
runAll(inputList)


2021-06-09 18:55:24,577 : Flow Logger HTML_Y : Starting HTML Conversion To Json | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:24,591 : Style Dictionary_8 : Reading style dictionary in file: rule_dictionary_ro.json | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:24,591 : Style Dictionary_8 : Reading style dictionary in file: rule_dictionary_ro.json | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:24,688 : Style Dictionary_8 : Qrd Section Keys Retrieved For Style Dictionary: REZUMATUL CARACTERISTICILOR PRODUSULUI, ANEXA II, A. ETICHETAREA, B. PROSPECTUL | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:24,688 : Style Dictionary_8 : Qrd Section Keys Retrieved For Style Dictionary: REZUMATUL CARACTERISTICILOR PRODUSULUI, ANEXA II, A. ETICHETAREA, B. PROSPECTUL | H | CAP |  ro | HTML | Elocta_clean.htm


Elocta~H~CAP~ro~2021-06-09T10-12-05Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'ro', '2021-06-09T10-12-05Z'] Elocta_clean.htm
2021-06-09T10-12-05Z ro Elocta CAP H
B. PROSPECTUL
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\outputJSON\Elocta_clean.json


2021-06-09 18:55:25,935 : Parser_K : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\outputJSON\Elocta_clean.txt | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:31,661 : Parser_K : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\outputJSON\Elocta_clean.json | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:32,214 : Flow Logger HTML_Y : Completed HTML Conversion To Json | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:32,216 : Flow Logger HTML_Y : HTML Conversion To Json,0.1273 Min,6.720758 MB,15.876118 MB,44.6%
 | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:32,230 : Flow Logger HTML_Y : Starting Json Split | H | CAP |  ro | HTML | Elocta_clean.htm
2021-06-09 18:55:32,235 : Style Dictionary_B : Reading style dictionary in file: rule_dictionary_ro.json | H | CAP |  ro | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1273 Min,6.720758 MB,15.876118 MB,44.6%

B. PROSPECTUL
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\outputJSON\Elocta_clean.json


2021-06-09 18:55:32,692 : Partition_M : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  ro | Json | Elocta_clean.json
2021-06-09 18:55:32,816 : Partition_M : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\partitionedJSONs\Elocta_clean_ANEXA II.json | H | CAP |  ro | Json | Elocta_clean.json
2021-06-09 18:55:32,962 : Partition_M : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\partitionedJSONs\Elocta_clean_ ETICHETAREA.json | H | CAP |  ro | Json | Elocta_clean.json
2021-06-09 18:55:32,991 : Partition_M : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\ro\2021-06-09T10-12-05Z\partitionedJSONs\Elocta_clean_ PROSPECTUL.json | H | CA

Metrics : Split Json,0.0131 Min,0.257973 MB,17.430485 MB,44.6%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'ro'

In [30]:
inputList = ['Elocta~H~CAP~sk~2021-06-09T10-12-51Z.zip']
runAll(inputList)


2021-06-09 18:55:59,036 : Flow Logger HTML_B : Starting HTML Conversion To Json | H | CAP |  sk | HTML | Elocta_clean.htm
2021-06-09 18:55:59,047 : Style Dictionary_4 : Creating default style dictionary in file: rule_dictionary_sk.json | H | CAP |  sk | HTML | Elocta_clean.htm
2021-06-09 18:55:59,117 : Style Dictionary_4 : Qrd Section Keys Retrieved For Style Dictionary: SÚHRN CHARAKTERISTICKÝCH VLASTNOSTÍ LIEKU, PRÍLOHA II, A. OZNAČENIE OBALU, B. PÍSOMNÁ INFORMÁCIA PRE POUŽÍVATEĽA | H | CAP |  sk | HTML | Elocta_clean.htm


Elocta~H~CAP~sk~2021-06-09T10-12-51Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'sk', '2021-06-09T10-12-51Z'] Elocta_clean.htm
2021-06-09T10-12-51Z sk Elocta CAP H
B. PÍSOMNÁ INFORMÁCIA PRE POUŽÍVATEĽA
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\outputJSON\Elocta_clean.json


2021-06-09 18:56:01,072 : Parser_D : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\outputJSON\Elocta_clean.txt | H | CAP |  sk | HTML | Elocta_clean.htm
2021-06-09 18:56:07,703 : Parser_D : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\outputJSON\Elocta_clean.json | H | CAP |  sk | HTML | Elocta_clean.htm
2021-06-09 18:56:08,229 : Flow Logger HTML_B : Completed HTML Conversion To Json | H | CAP |  sk | HTML | Elocta_clean.htm
2021-06-09 18:56:08,231 : Flow Logger HTML_B : HTML Conversion To Json,0.1533 Min,7.010127 MB,17.502162 MB,43.9%
 | H | CAP |  sk | HTML | Elocta_clean.htm
2021-06-09 18:56:08,244 : Flow Logger HTML_B : Starting Json Split | H | CAP |  sk | HTML | Elocta_clean.htm
2021-06-09 18:56:08,251 : Style Dictionary_6 : Reading style dictionary in file: rule_dictionary_sk.json | H | CAP |  sk | Json | Elocta_c

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1533 Min,7.010127 MB,17.502162 MB,43.9%

B. PÍSOMNÁ INFORMÁCIA PRE POUŽÍVATEĽA
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\outputJSON\Elocta_clean.json


2021-06-09 18:56:08,729 : Partition_R : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  sk | Json | Elocta_clean.json
2021-06-09 18:56:08,865 : Partition_R : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\partitionedJSONs\Elocta_clean_PRÍLOHA II.json | H | CAP |  sk | Json | Elocta_clean.json
2021-06-09 18:56:09,019 : Partition_R : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\partitionedJSONs\Elocta_clean_ OZNAČENIE OBALU.json | H | CAP |  sk | Json | Elocta_clean.json
2021-06-09 18:56:09,049 : Partition_R : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sk\2021-06-09T10-12-51Z\partitionedJSONs\Elocta_clean_ PÍSOMNÁ INFORMÁCIA

Metrics : Split Json,0.014 Min,0.213637 MB,18.465832 MB,43.8%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'sk'

In [31]:
inputList = ['Elocta~H~CAP~sl~2021-06-09T10-13-47Z.zip']
runAll(inputList)


2021-06-09 18:56:17,759 : Flow Logger HTML_t : Starting HTML Conversion To Json | H | CAP |  sl | HTML | Elocta_clean.htm
2021-06-09 18:56:17,772 : Style Dictionary_c : Creating default style dictionary in file: rule_dictionary_sl.json | H | CAP |  sl | HTML | Elocta_clean.htm
2021-06-09 18:56:17,772 : Style Dictionary_c : Creating default style dictionary in file: rule_dictionary_sl.json | H | CAP |  sl | HTML | Elocta_clean.htm


Elocta~H~CAP~sl~2021-06-09T10-13-47Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'sl', '2021-06-09T10-13-47Z'] Elocta_clean.htm
2021-06-09T10-13-47Z sl Elocta CAP H


2021-06-09 18:56:17,863 : Style Dictionary_c : Qrd Section Keys Retrieved For Style Dictionary: POVZETEK GLAVNIH ZNAČILNOSTI ZDRAVILA, PRILOGA II, A. OZNAČEVANJE, B. NAVODILO ZA UPORABO | H | CAP |  sl | HTML | Elocta_clean.htm
2021-06-09 18:56:17,863 : Style Dictionary_c : Qrd Section Keys Retrieved For Style Dictionary: POVZETEK GLAVNIH ZNAČILNOSTI ZDRAVILA, PRILOGA II, A. OZNAČEVANJE, B. NAVODILO ZA UPORABO | H | CAP |  sl | HTML | Elocta_clean.htm


B. NAVODILO ZA UPORABO
------------- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.txt -----------------
C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\Elocta_clean.htm C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.json


2021-06-09 18:56:19,408 : Parser_B : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.txt | H | CAP |  sl | HTML | Elocta_clean.htm
2021-06-09 18:56:19,408 : Parser_B : Style Information Stored In File: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.txt | H | CAP |  sl | HTML | Elocta_clean.htm
2021-06-09 18:56:25,198 : Parser_B : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.json | H | CAP |  sl | HTML | Elocta_clean.htm
2021-06-09 18:56:25,198 : Parser_B : Writing to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.json | H | CAP |  sl | HTML | Elocta_clean.htm
2021-06-09 18:56:25,721 : Flow Logger HTML_t : Compl

stylePath:- C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.txt
Metrics : HTML Conversion To Json,0.1328 Min,6.885995 MB,15.256466 MB,43.8%

B. NAVODILO ZA UPORABO
PathJson C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\outputJSON\Elocta_clean.json


2021-06-09 18:56:26,184 : Partition_C : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\partitionedJSONs\Elocta_clean_SmPC.json | H | CAP |  sl | Json | Elocta_clean.json
2021-06-09 18:56:26,321 : Partition_C : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\partitionedJSONs\Elocta_clean_PRILOGA II.json | H | CAP |  sl | Json | Elocta_clean.json
2021-06-09 18:56:26,461 : Partition_C : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\partitionedJSONs\Elocta_clean_ OZNAČEVANJE.json | H | CAP |  sl | Json | Elocta_clean.json
2021-06-09 18:56:26,495 : Partition_C : Writing partition to file: C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sl\2021-06-09T10-13-47Z\partitionedJSONs\Elocta_clean_ NAVODILO ZA UPORABO.js

Metrics : Split Json,0.0131 Min,0.196456 MB,17.00779 MB,43.8%

Starting Heading Extraction For File :- Elocta_clean_SmPC.json


KeyError: 'sl'

In [32]:
inputList = ['Elocta~H~CAP~sv~2021-06-09T10-14-39Z.zip']
runAll(inputList)


2021-06-09 18:56:32,969 : Flow Logger HTML_9 : Starting HTML Conversion To Json | H | CAP |  sv | HTML | Elocta_clean.htm
2021-06-09 18:56:32,969 : Flow Logger HTML_9 : Starting HTML Conversion To Json | H | CAP |  sv | HTML | Elocta_clean.htm
2021-06-09 18:56:32,991 : Style Dictionary_0 : Creating default style dictionary in file: rule_dictionary_sv.json | H | CAP |  sv | HTML | Elocta_clean.htm
2021-06-09 18:56:33,064 : Style Dictionary_0 : Language Code Not Found in QRD Template | H | CAP |  sv | HTML | Elocta_clean.htm
Traceback (most recent call last):
  File "C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\code\parse\rulebook\rulebook.py", line 108, in getSectionKeys
    raise LanguageErrorQrdTemplate("Language not found in QRD template")
parse.rulebook.rulebook.LanguageErrorQrdTemplate: Language not found in QRD template
2021-06-09 18:56:33,069 : Style Dictionary_0 : Language Code Not Found in QRD Template | H | CAP |  sv | HTML | Elocta_clean.htm


Elocta~H~CAP~sv~2021-06-09T10-14-39Z.zip C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\inputblob C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\work\H\CAP\Elocta\sv\2021-06-09T10-14-39Z C:\Users\psaga\source\repos\EMA\EMA%20EPI%20PoC\function_code\control
Already Present
Elocta_clean.htm
['C:', 'Users', 'psaga', 'source', 'repos', 'EMA', 'EMA%20EPI%20PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'sv', '2021-06-09T10-14-39Z'] Elocta_clean.htm
2021-06-09T10-14-39Z sv Elocta CAP H


LanguageErrorQrdTemplate: Language not found in QRD template